In [ ]:
"""
采用gluon实现AlexNet网络：2012年 ImageNet 冠军算法
本质：
    LeNet算法加强版
"""

from mxnet import gluon, nd, image

import sys
sys.path.append('..')
import utils

net = gluon.nn.Sequential()
with net.name_scope():
    # 第一阶段：一层卷积+一层池化
    net.add(gluon.nn.Conv2D(channels=96, kernel_size=11, strides=4, activation='relu'))
    net.add(gluon.nn.MaxPool2D(pool_size=3, strides=2))
    
    # 第二阶段：一层卷积+一层池化
    net.add(gluon.nn.Conv2D(channels=256, kernel_size=5, padding=2, activation='relu'))
    net.add(gluon.nn.MaxPool2D(pool_size=3, strides=2))
    
    # 第三阶段：三层卷积+一层池化
    net.add(gluon.nn.Conv2D(channels=384, kernel_size=3, padding=1, activation='relu'))
    net.add(gluon.nn.Conv2D(channels=384, kernel_size=3, padding=1, activation='relu'))
    net.add(gluon.nn.Conv2D(channels=256, kernel_size=3, padding=1, activation='relu'))
    net.add(gluon.nn.MaxPool2D(pool_size=3, strides=2))
    
    # 第四阶段：flatten扁平化 + Dense全连接 + dropout正则化
    net.add(gluon.nn.Flatten())
    net.add(gluon.nn.Dense(4096, activation='relu'))
    net.add(gluon.nn.Dropout(0.5))
    
    # 第五阶段：Dense全

连接 + dropout正则化
    net.add(gluon.nn.Dense(4096, activation='relu'))
    net.add(gluon.nn.Dropout(0.5))
    
    # 第六阶段：Dense层输出
    net.add(gluon.nn.Dense(10))


In [ ]:
# 对输入数据进行转换，保存为float32格式
def transform(data, label):
    # resize from 28 * 28 to 224 * 224
    data = image.imresize(data, 224, 224)
    return utils.load_data_fashion_mnist.transform_mnist(data, label)

In [47]:
batch_size = 64
train_data, test_data = utils.load_data_fashion_mnist(batch_size)
print(train_data)

NameError: name 'utils' is not defined

In [60]:
for data, label in train_data:
    print(data.shape)
    print(label.shape)

MXNetError: Invalid Parameter format for shape expect Shape(tuple) but value='(64, <function transform at 0x7ff62af1e620>, <function transform at 0x7ff62af1e620>, 1)', in operator _zeros(name="", dtype="float32", ctx="cpu(0)", shape="(64, <function transform at 0x7ff62af1e620>, <function transform at 0x7ff62af1e620>, 1)")

In [58]:
"""
开始训练：
    1. 使用Xavier初始化参数；
    2. 使用更小学习率
    
"""

from mxnet import autograd
from mxnet import gluon
from mxnet import nd
from mxnet import init

ctx = utils.try_gpu()
net.initialize(ctx=ctx, init=init.Xavier())

learning_rate = 0.01
epochs = 1

softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': learning_rate})

for epoch in range(epochs):
    train_loss = 0.
    test_loss = 0.
    
    for data, label in train_data:
        with autograd.record():
            output = net(data.as_in_context(ctx))  #数据集迁移只ctx设备
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(batch_size)
        
        train_loss += nd.mean(loss).asscalar()
        train_acc = utils.accuracy(output, label)
        
    test_acc = utils.evaluate_accuracy(test_data, label)
    print('Epoch: %d.\nTrain loss: %f.\nTrain acc: %f.\nTest acc: %f' % \
          (epoch, train_loss / len(train_data), train_acc / len(train_data), test_acc))

/home/frank/anaconda3/envs/gluon/lib/python3.6/site-packages/mxnet/gluon/parameter.py:687: UserWarning: Parameter 'sequential4_conv0_bias' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)
/home/frank/anaconda3/envs/gluon/lib/python3.6/site-packages/mxnet/gluon/parameter.py:687: UserWarning: Parameter 'sequential4_conv1_bias' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)
/home/frank/anaconda3/envs/gluon/lib/python3.6/site-packages/mxnet/gluon/parameter.py:687: UserWarning: Parameter 'sequential4_conv2_bias' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)
/home/frank/anaconda3/envs/gluon/lib/python3.6/site-packages/mxnet/gluon/parameter.py:687: UserWarning: Parameter 'sequential4_conv3_bias' is already initialized, ignoring. Set force_r

MXNetError: Invalid Parameter format for shape expect Shape(tuple) but value='(64, <function transform at 0x7ff62af1e620>, <function transform at 0x7ff62af1e620>, 1)', in operator _zeros(name="", dtype="float32", ctx="cpu(0)", shape="(64, <function transform at 0x7ff62af1e620>, <function transform at 0x7ff62af1e620>, 1)")

In [33]:
def load_data_fashion_mnist(batch_size, resize=None, root="~/.mxnet/datasets/fashion-mnist"):
    """download the fashion mnist dataest and then load into memory"""
    def transform_mnist(data, label):
        # Transform a batch of examples.
        if resize:
            n = data.shape[0]
            new_data = nd.zeros((n, resize, resize, data.shape[3]))
            for i in range(n):
                new_data[i] = image.imresize(data[i], resize, resize)
            data = new_data
        # change data from batch x height x width x channel to batch x channel x height x width
        return nd.transpose(data.astype('float32'), (0,3,1,2))/255, label.astype('float32')

    mnist_train = gluon.data.vision.FashionMNIST(root=root, train=True, transform=None)
    mnist_test = gluon.data.vision.FashionMNIST(root=root, train=False, transform=None)
    # Transform later to avoid memory explosion. 
    train_data = DataLoader(mnist_train, batch_size, shuffle=True, transform=transform_mnist)
    test_data = DataLoader(mnist_test, batch_size, shuffle=False, transform=transform_mnist)
    return (train_data, test_data)

def try_gpu():
    """If GPU is available, return mx.gpu(0); else return mx.cpu()"""
    try:
        ctx = mx.gpu()
        _ = nd.array([0], ctx=ctx)
    except:
        ctx = mx.cpu()
    return ctx


In [17]:
data_set = gluon.data.vision.FashionMNIST()

In [42]:
train_data = gluon.data.DataLoader(data_set, 100)

In [46]:
cnt = 0
for data,label in train_data:
    #print('data is: ', len(data))
    cnt += 1
print('cnt is: ', cnt)
print('data[0] is: ', data[0].shape)
print('label.shape is: ', label.shape)
print('label[0].shape is: ', label[0].shape)

cnt is:  6000
data[0] is:  (28, 28, 1)
label.shape is:  (10,)
label[0].shape is:  (1,)
